# [Tutorial](https://github.com/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_NeurIPS2018/Stock_NeurIPS2018_2_Train.ipynb)

# 1.Importando as bibliotecas

In [1]:
#!python --version

In [2]:
#!pip list > pacotes_instalados_colab.txt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.6 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.23
    Uninstalling SQLAlchemy-2.0.23:
      Successfully uninstalled SQLAlchemy-2.0.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.50 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 962.5 kB/s eta 0:00:00
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-ixrb9xrp
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-ixrb9xrp
  Resolved h

In [5]:
import os
import pandas as pd

from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl import config_tickers
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR

## Cria o diretorio TRAINED_MODEL_DIR
check_and_make_directories([TRAINED_MODEL_DIR])

In [6]:
import time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 2.Importando os dados

Os dados importados são do primeiro notebook [data_YahooDownloader.ipynb](Documents/Aprendizado_reforco_trading/Finrl/FinRL_testes/data_YahooDownloader.ipynb)

In [7]:
try:
  treino = pd.read_csv('train_data_ibovespa.csv')
  treino = treino.set_index(treino.columns[0])
  treino.index.names = ['']
except:
  treino = pd.read_csv('/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/FinRL_testes/trade_data_ibovespa.csv')
  treino = treino.set_index(treino.columns[0])
  treino.index.names = ['']


In [8]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28440 entries, 0 to 631
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          28440 non-null  object 
 1   tic           28440 non-null  object 
 2   open          28440 non-null  float64
 3   high          28440 non-null  float64
 4   low           28440 non-null  float64
 5   close         28440 non-null  float64
 6   volume        28440 non-null  float64
 7   day           28440 non-null  float64
 8   macd          28440 non-null  float64
 9   boll_ub       28440 non-null  float64
 10  boll_lb       28440 non-null  float64
 11  rsi_30        28440 non-null  float64
 12  cci_30        28440 non-null  float64
 13  dx_30         28440 non-null  float64
 14  close_30_sma  28440 non-null  float64
 15  close_60_sma  28440 non-null  float64
 16  vix           28440 non-null  float64
 17  turbulence    28440 non-null  float64
dtypes: float64(16), object(2)
me

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
treino.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
,,,,,,,,,,,,,,,,,,
0,2019-06-19,ABEV3.SA,18.049999,18.360001,17.980000,15.721224,23446200.0,2.0,0.152904,15.663305,14.661381,57.200970,167.722872,23.650660,14.982096,15.036227,14.33,24.913164
0,2019-06-19,BBAS3.SA,51.500000,52.500000,51.110001,39.596638,7964300.0,2.0,0.529207,40.401258,37.154286,56.015006,66.083933,2.784967,37.781369,37.048178,14.33,24.913164
0,2019-06-19,BBDC3.SA,24.350113,24.748308,23.944403,19.880020,3485755.0,2.0,0.282393,20.480991,18.284890,56.957863,67.945098,12.518559,18.834974,18.733100,14.33,24.913164
0,2019-06-19,BBDC4.SA,27.227648,27.723516,26.867018,22.328823,20888447.0,2.0,0.209542,23.129236,20.960027,54.459579,52.308731,9.690325,21.571333,21.516488,14.33,24.913164
0,2019-06-19,BEEF3.SA,7.866358,7.954854,7.797527,6.243949,6414658.0,2.0,0.037579,6.432561,6.092708,53.460961,-20.314667,15.627085,6.255368,6.108482,14.33,24.913164


# 3.Criando o ambiente

O [ambiente](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/env_stock_trading/env_stocktrading.py) de negociação de ações criado para ser utilizado com a biblioteca OpenAI Gym

In [10]:
valor_inicial_carteira = 1000
custo_trade = [0.01]
taxa_aprendizagem = 0.01
iteracao = 1000000

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
stock_dimension = len(treino.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 45, State Space: 451


In [12]:
buy_cost_list = sell_cost_list = custo_trade * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount":valor_inicial_carteira ,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = treino, **env_kwargs)

In [13]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# 4.Treinando os modelos
Serao treinados 5 modelos da biblioteca **[Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/)**

Os modelos de aprendizagem de reforco são o: [A2C](https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html), [DDPG](https://stable-baselines3.readthedocs.io/en/master/modules/ddpg.html), [PPO](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html), [TD3](https://stable-baselines3.readthedocs.io/en/master/modules/td3.html), [SAC](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html).

## Configurações do ambiente

In [14]:
agent = DRLAgent(env = env_train)

# Colque true para o algoritimo que vc quer treinar
if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = True
if_using_sac = False

## Treinando o algoritimo A2C

In [15]:
agent = DRLAgent(env = env_train)
A2C_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": taxa_aprendizagem
}
model_a2c = agent.get_model("a2c",model_kwargs=A2C_PARAMS)


if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.01}
Using cpu device


In [16]:
start_time = time.time()
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=iteracao) if if_using_a2c else None
end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")

Tempo de execução: 0.00 minutos


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
trained_a2c.save(TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None

## Treinando o algoritimo DDPG

In [18]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {
    "learning_rate": taxa_aprendizagem,
    "batch_size": 128,
}
model_ddpg = agent.get_model("ddpg",model_kwargs=DDPG_PARAMS)

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'learning_rate': 0.01, 'batch_size': 128}
Using cpu device


In [19]:
start_time = time.time()

trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=iteracao-500000) if if_using_ddpg else None
end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")

Tempo de execução: 0.00 minutos


In [20]:
trained_ddpg.save("/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/FinRL_testes/trained_models" + "/agent_ddpg") if if_using_ddpg else None

In [21]:
trained_ddpg.save(TRAINED_MODEL_DIR + "/agent_ddpg") if if_using_ddpg else None

## Treinando o algoritimo PPO

In [22]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": taxa_aprendizagem,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.01, 'batch_size': 128}
Using cpu device


In [23]:
start_time = time.time()
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=iteracao) if if_using_ppo else None

end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")

Tempo de execução: 0.00 minutos


In [24]:
trained_ddpg.save("/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/FinRL_testes/trained_models" + "/agent_ppo") if if_using_ddpg else None

In [25]:
trained_ppo.save(TRAINED_MODEL_DIR + "/agent_ppo") if if_using_ppo else None

## Treinando o algoritimo TD3

In [26]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": taxa_aprendizagem}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.01}
Using cpu device
Logging to results/td3


In [27]:
start_time = time.time()
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=iteracao-600000) if if_using_td3 else None
end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")


----------------------------------------
| time/              |                 |
|    episodes        | 4               |
|    fps             | 26              |
|    time_elapsed    | 96              |
|    total_timesteps | 2528            |
| train/             |                 |
|    actor_loss      | -3.45           |
|    critic_loss     | 0.494           |
|    learning_rate   | 0.01            |
|    n_updates       | 1896            |
|    reward          | -0.000101107595 |
----------------------------------------
----------------------------------------
| time/              |                 |
|    episodes        | 8               |
|    fps             | 22              |
|    time_elapsed    | 220             |
|    total_timesteps | 5056            |
| train/             |                 |
|    actor_loss      | -2.88           |
|    critic_loss     | 0.0767          |
|    learning_rate   | 0.01            |
|    n_updates       | 4424            |
|    reward     

In [28]:
TRAINED_MODEL_DIR

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'trained_models'

In [29]:
trained_td3.save(TRAINED_MODEL_DIR + "/agent_td3") if if_using_td3 else None

## Treinando o algoritimo SAC

In [30]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": taxa_aprendizagem,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.01, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [31]:
start_time = time.time()
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=iteracao) if if_using_sac else None
end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")

Tempo de execução: 0.00 minutos


In [32]:
trained_sac.save(TRAINED_MODEL_DIR + "/agent_sac") if if_using_sac else None